In [2]:
import pandas as pd
import mysql.connector

In [ ]:
# Cargar CSV´s

df_balance = pd.read_csv("../data/processed/balance/balance-electrico-limpio.csv")

df_demanda_evolucion = pd.read_csv("../data/processed/demanda/demanda-limpio.csv")
df_demanda_ire = pd.read_csv("../data/processed/demanda/ire-limpio.csv")

df_estructura_generacion = pd.read_csv("../data/processed/generacion/estructura-generacion-limpio.csv")

df_fronteras = pd.read_csv("../data/processed/intercambio/fronteras-limpio.csv")

In [18]:
df_balance.head()

,fecha,valor,tipo,energia,region
0,2019-01-01,53.722120,Hidráulica,Renovable,peninsular
1,2019-01-02,61.294768,Hidráulica,Renovable,peninsular
2,2019-01-03,82.981951,Hidráulica,Renovable,peninsular
3,2019-01-04,94.301940,Hidráulica,Renovable,peninsular
4,2019-01-05,58.822692,Hidráulica,Renovable,peninsular


In [20]:
df_demanda_evolucion.head()

,fecha,valor,indicador,region,año
0,2019-01-01,582.949806,Demanda,peninsular,2019
1,2019-01-02,742.199407,Demanda,peninsular,2019
2,2019-01-03,787.764963,Demanda,peninsular,2019
3,2019-01-04,796.010035,Demanda,peninsular,2019
4,2019-01-05,721.029651,Demanda,peninsular,2019


In [21]:
df_demanda_ire.head()

,fecha,valor,porcentaje,indicador,region,año
0,2019-01-01,123.969,0.541252,Índice general,peninsular,2019
1,2019-02-01,114.805,0.516888,Índice general,peninsular,2019
2,2019-03-01,126.515,0.541843,Índice general,peninsular,2019
3,2019-04-01,119.432,0.532477,Índice general,peninsular,2019
4,2019-05-01,130.280,0.552359,Índice general,peninsular,2019


In [22]:
df_estructura_generacion.head()

,fecha,valor,porcentaje,indicador,region,tipo,año
0,2019-01-01,53.722120,0.097735,Hidráulica,peninsular,Renovable,2019
1,2019-01-02,61.294768,0.086331,Hidráulica,peninsular,Renovable,2019
2,2019-01-03,82.981951,0.115989,Hidráulica,peninsular,Renovable,2019
3,2019-01-04,94.301940,0.125121,Hidráulica,peninsular,Renovable,2019
4,2019-01-05,58.822692,0.086633,Hidráulica,peninsular,Renovable,2019


In [19]:
df_fronteras.head()

,valor,porcentaje,fecha,pais,año
0,-0.208239,0.003465,2019-01-01,francia-frontera,2019
1,-2.325553,0.036857,2019-01-02,francia-frontera,2019
2,-11.123719,0.243767,2019-01-03,francia-frontera,2019
3,-17.758093,0.397978,2019-01-04,francia-frontera,2019
4,-4.267317,0.102386,2019-01-05,francia-frontera,2019


In [ ]:
# Conexión MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password='rootroot',
    database="redes_electricas"
)

cursor = conn.cursor()

In [61]:
nuevo_balance = ['fecha', 'tipo', 'energia', 'region', 'valor']
df_balance = df_balance[nuevo_balance]

df_balance.head()

,fecha,tipo,energia,region,valor
0,2019-01-01,Hidráulica,Renovable,peninsular,53.722120
1,2019-01-02,Hidráulica,Renovable,peninsular,61.294768
2,2019-01-03,Hidráulica,Renovable,peninsular,82.981951
3,2019-01-04,Hidráulica,Renovable,peninsular,94.301940
4,2019-01-05,Hidráulica,Renovable,peninsular,58.822692


In [ ]:
# Subir datos balance
for _, row in df_balance.iterrows():
    cursor.execute("""
        INSERT IGNORE INTO balance (fecha, tipo, energia, region, valor)
        VALUES (%s, %s, %s, %s, %s)
    """, (row['fecha'], row['tipo'], row['energia'], row['region'], row['valor']))

conn.commit()

In [ ]:
nuevo_ire = ['fecha', 'indicador', 'region', 'valor', 'porcentaje', 'año']
df_demanda_ire = df_demanda_ire[nuevo_ire]

df_demanda_ire = df_demanda_ire.drop('año', axis=1)

df_demanda_ire.head()

,fecha,indicador,region,valor,porcentaje
0,2019-01-01,Índice general,peninsular,123.969,0.541252
1,2019-02-01,Índice general,peninsular,114.805,0.516888
2,2019-03-01,Índice general,peninsular,126.515,0.541843
3,2019-04-01,Índice general,peninsular,119.432,0.532477
4,2019-05-01,Índice general,peninsular,130.280,0.552359


In [ ]:
# Subir datos demanda ire

for _, row in df_demanda_ire.iterrows():
    cursor.execute("""
        INSERT IGNORE INTO demanda_ire_general (fecha, indicador, region, valor, porcentaje)
        VALUES (%s, %s, %s, %s, %s)
    """, (row['fecha'], row['indicador'], row['region'], row['valor'], row['porcentaje']))

conn.commit()

In [45]:
nuevo_demanda = ['fecha', 'indicador', 'region', 'valor']
df_demanda_evolucion = df_demanda_evolucion[nuevo_demanda]
df_demanda_evolucion.head()

,fecha,indicador,region,valor
0,2019-01-01,Demanda,peninsular,582.949806
1,2019-01-02,Demanda,peninsular,742.199407
2,2019-01-03,Demanda,peninsular,787.764963
3,2019-01-04,Demanda,peninsular,796.010035
4,2019-01-05,Demanda,peninsular,721.029651


In [ ]:
# subir datos demanda evolucion

for _, row in df_demanda_evolucion.iterrows():
    cursor.execute("""
        INSERT IGNORE INTO demanda_evolucion (fecha, indicador, region, valor)
        VALUES (%s, %s, %s, %s)
    """, (row['fecha'], row['indicador'], row['region'], row['valor']))

conn.commit()

In [67]:
nuevo_fronteras = ['fecha', 'pais', 'valor', 'porcentaje', 'año']
df_fronteras = df_fronteras[nuevo_fronteras]
df_fronteras = df_fronteras.drop('año', axis=1)
df_fronteras.head()

,fecha,pais,valor,porcentaje
0,2019-01-01,francia-frontera,-0.208239,0.003465
1,2019-01-02,francia-frontera,-2.325553,0.036857
2,2019-01-03,francia-frontera,-11.123719,0.243767
3,2019-01-04,francia-frontera,-17.758093,0.397978
4,2019-01-05,francia-frontera,-4.267317,0.102386


In [ ]:
# Subir datos itercambio
for _, row in df_fronteras.iterrows():
    cursor.execute("""
        INSERT IGNORE INTO fronteras (fecha, pais, valor, porcentaje)
        VALUES (%s, %s, %s, %s)
    """, (row['fecha'], row['pais'], row['valor'], row['porcentaje']))

conn.commit()

In [75]:
nuevo_estructura = ['fecha', 'indicador', 'region', 'tipo', 'valor', 'porcentaje', 'año']
df_estructura_generacion = df_estructura_generacion[nuevo_estructura]
df_estructura_generacion = df_estructura_generacion.drop('año', axis=1)
df_estructura_generacion.head()

,fecha,indicador,region,tipo,valor,porcentaje
0,2019-01-01,Hidráulica,peninsular,Renovable,53.722120,0.097735
1,2019-01-02,Hidráulica,peninsular,Renovable,61.294768,0.086331
2,2019-01-03,Hidráulica,peninsular,Renovable,82.981951,0.115989
3,2019-01-04,Hidráulica,peninsular,Renovable,94.301940,0.125121
4,2019-01-05,Hidráulica,peninsular,Renovable,58.822692,0.086633


In [100]:
# Subir datos generacion
for _, row in df_estructura_generacion.iterrows():
    cursor.execute("""
        INSERT IGNORE INTO estructura_generacion (fecha, indicador, region, tipo, valor, porcentaje)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (row['fecha'], row['indicador'], row['region'], row['tipo'], row['valor'], row['porcentaje']))

conn.commit()
cursor.close()
conn.close()